## 机器学习模型训练流程

### 1. 数据的导入与显示


In [3]:
#检查python版本代码
#sys是系统学习
#sys.version_info  sys.version_info(2, 4, 3, 'final', 0) 
#'final'表示最终,也有'candidate'表示候选，表示版本级别，是否有后继的发行
#major表示主版本号，minor表示次版本号
from sys import version_info
if version_info.major != 2 and version_info.minor != 7:
    raise Exception('请使用Python 2.7来完成此项目')
else:
    print version_info

sys.version_info(major=2, minor=7, micro=12, releaselevel='final', serial=0)


In [4]:
import numpy as np
import pandas as pd

# 数据可视化代码
from titanic_visualizations import survival_stats
from IPython.display import display
%matplotlib inline
#用户代码在Kernel中的shell对象中执行之后，通过DisplayFormatter将需要显示的对象转换为一个描述显示信息的字典，
#并传递给客户端进行显示。目前IPython有三种客户端：控制台、QtConsole和Notebook。
#每种客户端都会从这个字典中提取自己能处理的信息进行显示。

# 加载数据集
in_file = 'titanic_data.csv'
full_data = pd.read_csv(in_file) #读取csv文件

# 显示数据列表中的前几项乘客数据
#a.head(6)表示显示前6行数据，若head()中不带参数则会显示全部数据。
#a.tail(6)表示显示后6行数据，若tail()中不带参数则也会显示全部数据。
display(full_data.head())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
## 分离结果(标签)与特征
labels_raw = full_data['Survived'] # 即labes
features_raw = full_data.drop('Survived',axis=1) #即features

### 2. 分析数据，计算统计信息

In [ ]:
#此处以波士顿房价预测为例
#目标：计算价值的最小值
minimum_price = np.min(prices)

#目标：计算价值的最大值
maximum_price = np.max(prices)

#目标：计算价值的平均值
mean_price = np.mean(prices)

#目标：计算价值的中值
median_price = np.median(prices)

#目标：计算价值的标准差
std_price = np.std(prices)

#目标：输出计算的结果
print "Statistics for Boston housing dataset:\n"
print "Minimum price: ${:,.2f}".format(minimum_price)
print "Maximum price: ${:,.2f}".format(maximum_price)
print "Mean price: ${:,.2f}".format(mean_price)
print "Median price ${:,.2f}".format(median_price)
print "Standard deviation of prices: ${:,.2f}".format(std_price)

### 3. 拆分数据，创造训练集与测试集(若不需要预处理此时即可拆分)

In [4]:
from sklearn.model_selection  import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, outcomes, test_size = 0.2, 
                                                    random_state =0)

### 4.定义模型衡量标准

(1)[*决定系数*](http://stattrek.com/statistics/dictionary.aspx?definition=coefficient_of_determination) R<sup>2</sup> 用于量化模型的表现。它是回归分析中十分常用的统计信息，经常被当作衡量模型预测能力好坏的标准。

R<sup>2</sup>的数值范围从0至1，表示**目标变量**的预测值和实际值之间的相关程度平方的百分比。一个模型的R<sup>2</sup> 值为0还不如直接用**平均值**来预测效果好；而一个R<sup>2</sup> 值为1的模型则可以对目标变量进行完美的预测。从0至1之间的数值，则表示该模型中目标变量中有百分之多少能够用**特征**来解释。_模型也可能出现负值的R<sup>2</sup>，这种情况下模型所做预测有时会比直接计算目标变量的平均值差很多。_


In [7]:
## 用sklearn计算决定系数
def performance_metric(y_true, y_predict):
    """计算并返回预测值相比于预测值的分数"""
    from sklearn.metrics import r2_score
    score = r2_score(y_true, y_predict)
    return score

## 此处补充其他评价指标:F分数，精确度
from sklearn.metrics import fbeta_score, accuracy_score

acc = accuracy_score(y_true, y_predict)
    
f_score = fbeta_score(y_true, y_predict, beta=0.5)

In [ ]:
## 根据公式自己计算决定系数
def performance_metric2(y_true, y_predict):
    """计算并返回预测值相比于预测值的分数"""
    yse = np.array(y_true)-np.array(y_predict)
    sse = np.dot(yse, yse)
    y_mean = np.mean(y_true)
    yst = np.array(y_true)-np.array(y_mean)
    sst = np.dot(yst, yst)   
    score = 1-sse/sst
    return score

#### 学习曲线的书写

### 5.预处理
（1）偏斜特征处理：取对数

一个数据集有时可能包含至少一个靠近某个数字的特征，但有时也会有一些相对来说存在极大值或者极小值的不平凡分布的特征。算法对这种分布的数据会十分敏感，并且如果这种数据没有能够很好地规一化处理会使得算法表现不佳。 对于高度倾斜分布的特征，常见的做法是对数据施加一个对数转换，将数据转换成对数，这样非常大和非常小的值不会对学习算法产生负面的影响。并且使用对数变换显著降低了由于异常值所造成的数据范围异常。但是在应用这个变换时必须小心：因为0的对数是没有定义的，所以我们必须先将数据处理成一个比0稍微大一点的数以成功完成对数转换，在其他数远大于1时可考虑+1，因为其他数远远大于1，加了之后不影响。

In [ ]:
skewed = ['feature_name1', 'feature_name2', ...]
features_raw[skewed] = full_data[skewed].apply(lambda x: np.log(x+1) )

（2）数字特征归一化

规一化保证了每一个特征在使用监督学习器的时候能够被平等的对待。缩放不会改变数据原始的分布，但注意一旦使用了缩放，观察数据的原始形式不再具有它本来的意义了。

In [ ]:
##可用工具包sklearn的MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
numerical = ['feature_name1', 'feature_name2', ...]
features_raw[numerical] = scaler.fit_transform(data[numerical])
# fit(x,y)传两个参数的是有监督学习的算法，fit(x)传一个参数的是无监督学习的算法，比如降维、特征提取、标准化
# fit_transform()的作用就是先拟合数据，找到该part的整体指标，如均值、方差、最大值最小值等，然后转化它将其转化为标准形式
# tranform()的作用是通过找中心和缩放等实现标准化
# 必须先用fit_transform(partData)，之后再transform(restData)，如果直接transform(partData)，程序会报错
# 可以对训练集fit_transform,然后对测试集transform,
# transform()可以直接把转换规则拿来用，所以并不需要fit_transform()，否则，两次标准化后的数据格式就不一样

（3）转换非数值特征：独热编码

通常情况下，学习算法期望输入是数字的，这要求非数字的特征（称为类别变量）被转换。转换类别变量的一种流行的方法是使用**独热编码**方案。独热编码为每一个非数字特征的每一个可能的类别创建一个_“虚拟”_变量。例如，假设`someFeature`有三个可能的取值`A`，`B`或者`C`，。我们将把这个特征编码成`someFeature_A`, `someFeature_B`和`someFeature_C`.

| 特征X |                    | 特征X_A | 特征X_B | 特征X_C |
| :-: |                            | :-: | :-: | :-: |
|  B  |  | 0 | 1 | 0 |
|  C  | ----> 独热编码 ----> | 0 | 0 | 1 |
|  A  |  | 1 | 0 | 0 |
- 使用[`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies)对`'features_raw'`数据来施加一个独热编码。

In [7]:
#分别对特征以及标签进行独热编码
## 1. 对特征进行独热编码: pd.get_dummies会把输入的非数字特征都编码
features = pd.get_dummies(features_raw)
# 查看display(features.head(5))

## 2. 对标签编码：
#labels = pd.get_dummies(labels_raw) 
#注意：独热编码是特征有几个可取的值出来就是几位的码，所以对于标签来说，想要的是0，1值，而不是10和01这样的码
#因此以下方法更为合适
income = income_raw.str.count('>50K')
#str.count()遍历income_raw列表，与>50一样的字符串置1，不一样置0
#income = income_raw.replace(['>50K', '<=50K'], [1, 0])

NameError: name 'features_raw' is not defined

（4）数据混洗和切分，预处理后的训练集/测试集/验证集划分

In [ ]:
# 导入 train_test_split
from sklearn.model_selection import train_test_split

# 将'features'和'income'数据切分成训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(features, income, test_size = 0.2, random_state = 0,
                                                    stratify = income)
# 将'X_train'和'y_train'进一步切分为训练集和验证集
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0,
                                                    stratify = y_train)
# 此时的训练集已经不包括验证集了

# 显示切分的结果
print "Training set has {} samples.".format(X_train.shape[0])
print "Validation set has {} samples.".format(X_val.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

### 8. 模型训练

#### 交叉验证
1. 在K折交叉验证中，用到的数据是训练集中的所有数据，过程是将训练集的所有数据平均划分成K份，取第K份作为验证集，余下的K-1份作为交叉验证的训练集。Kfold默认是把数据按顺序切分成K份，而当设置shuffle = True时，是将数据打乱后再切分。
2. GridSearchCV是将参数构成网格状的组合，每一次交叉验证选取其中的一组参数，直至所有参数全部被验证，比较所有参数组合结果，选取模型最优时的参数组合。
3. 多个模型的话，先比较各模型的训练精度，选出最好的模型，（这一步时，训练数据用训练集，比较精度时可用验证集），再用选出的模型进行交叉验证调参，如P2，寻找捐助者项目比较典型

In [ ]:
# 导入依赖包
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.grid_search import GridSearchCV
from sklearn import metrics
import numnpy as np
import pandas as pd

# 设置参数
params={'learning_rate':np.linspace(0.05,0.25,5), 
        'max_depth':[x for x in range(1,8,1)], 
        'min_samples_leaf':[x for x in range(1,5,1)],
        'n_estimators':[x for x in range(50,100,10)]}

#设置模型和评价指标，训练
clf = GradientBoostingClassifier()
grid = GridSearchCV(clf, params, cv=10, scoring="f1")#f1为评价标准函数
grid.fit(X, y)

# 查看最佳分数
grid.best_score_    #查看最佳分数(此处为f1_score)
grid.best_params_   #查看最佳参数

# 获取最佳分数
grid.best_estimator_

#用最佳分数预测
best_model=grid.best_estimator_
predict_y=best_model.predict(Test_X)
metrics.f1_score(y, predict_y)

In [ ]:
### 例子：交叉验证选择最优模型（决策树）
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV

def fit_model(X,y):
 
    cross_validator = KFold(10) #创建交叉验证生成器

    regressor = DecisionTreeRegressor() #创建决策树回归函数

    params = {"max_depth": range(1,11)}

    scoring_fnc = make_scorer(performance_metric)

    grid = GridSearchCV(regressor, params, scoring_fnc, cv=cross_validator)
    
    # 基于输入数据 [X,y],执行拟合和预测数据的试验。
    grid = grid.fit(X, y)
    #print pd.DataFrame(grid.cv_results_)
    # 返回网格搜索后的最优模型
    return grid.best_estimator_

In [ ]:
## 简易版交叉验证
from sklearn import cross_validation
features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(
    features, labels, test_size=0.3, random_state=42)

# 调用例子
clf = svm.SVC(kernel='linear', C=1)
scores = cross_validation.cross_val_score(clf, raw data, raw target, cv=5)

### 9.模型调整及评价

#### (1) 特征选择

In [ ]:
## 从最佳的模型中提取出前几个特征
model = ensemble.AdaBoostClassifier(random_state = 0)
model.fit(X_train, y_train)
# 提取特征重要性（一般要求前几个特征的和达到百分之多少）
importances = model.feature_importances_

# 绘图
vs.feature_plot(importances, X_train, y_train)

#### （2）用选出的特征重新训练模型
训练结果不一定比全特征的好，但是可以作为一个参考

In [ ]:
# 导入克隆模型的功能
from sklearn.base import clone

# 减小特征空间
X_train_reduced = X_train[X_train.columns.values[(np.argsort(importances)[::-1])[:5]]]
X_val_reduced = X_val[X_val.columns.values[(np.argsort(importances)[::-1])[:5]]]

# 在前面的网格搜索的基础上训练一个“最好的”模型
clf_on_reduced = (clone(best_clf)).fit(X_train_reduced, y_train)
##clone用于构造具有相同参数的新估计器

# 做一个新的预测
reduced_predictions = clf_on_reduced.predict(X_val_reduced)

# 对于每一个版本的数据汇报最终模型的分数
print "Final Model trained on full data\n------"
print "Accuracy on validation data: {:.4f}".format(accuracy_score(y_val, best_predictions))
print "F-score on validation data: {:.4f}".format(fbeta_score(y_val, best_predictions, beta = 0.5))
print "\nFinal Model trained on reduced data\n------"
print "Accuracy on validation data: {:.4f}".format(accuracy_score(y_val, reduced_predictions))
print "F-score on validation data: {:.4f}".format(fbeta_score(y_val, reduced_predictions, beta = 0.5))

#### （3）用测试集测试选出的最佳模型

In [ ]:
best_predictions = best_clf.predict(X_test)
accuracy = accuracy_score(y_test, best_predictions)
score = fbeta_score(y_test, best_predictions, beta=0.5)
print accuracy, score